In [1]:
import pandas as pd
from collections import defaultdict

def preprocess_data(df):
    transactions = []
    
    selected_columns = ['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Bwd Packet Length Max', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Fwd IAT Mean', 'Bwd IAT Mean', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'Average Packet Size', 'Subflow Fwd Packets', 'Subflow Bwd Packets', 'Active Mean', 'Idle Mean', 'Label']
    for index, row in df.iterrows():
        transaction = []
        for col in selected_columns:
            value = str(row[col])
            item = f"{col}:{value}"
            transaction.append(item)
        transactions.append(transaction)
    return transactions


In [2]:
class FPNode:
    def __init__(self, item, count, parent):
        self.item = item          # Например, "Protocol:TCP"
        self.count = count        # Подсчет вхождений
        self.parent = parent      # Ссылка на родительский узел
        self.children = {}        # Дочерние узлы: ключ - имя элемента, значение - объект FPNode
        self.node_link = None     # Ссылка на следующий узел с таким же элементом

    def increment(self, count): #Увеличивание счётчика
        self.count += count

def update_header(node, target_node): #добавляет новый узел в цепочку для элемента.
    while node.node_link is not None:
        node = node.node_link
    node.node_link = target_node

def update_tree(items, tree, header_table, count): #добавление отсортированного списка в FP-tree, если элемент есть то ++, если нет то добавляется новый в дочерний список
    first_item = items[0]
    if first_item in tree.children:
        tree.children[first_item].increment(count)
    else:
        new_node = FPNode(first_item, count, tree)
        tree.children[first_item] = new_node
        if header_table[first_item][1] is None:
            header_table[first_item][1] = new_node
        else:
            update_header(header_table[first_item][1], new_node)
    if len(items) > 1:
        update_tree(items[1:], tree.children[first_item], header_table, count)

def create_fp_tree(transactions, min_support):
    """
    Создает FP-дерево из транзакций.
    
    Алгоритм:
      1. Подсчитываем поддержку каждого элемента.
      2. Отбрасываем элементы с поддержкой ниже min_support.
      3. Формируем заголовочную таблицу: ключ – элемент, значение – [поддержка, ссылка на первый узел].
      4. Для каждой транзакции оставляем только частые элементы,
         сортированные по убыванию поддержки.
      5. Рекурсивно добавляем транзакцию в дерево.
    """
    freq = {}
    for transaction in transactions:
        for item in transaction:
            freq[item] = freq.get(item, 0) + 1
    freq = {item: count for item, count in freq.items() if count >= min_support}
    if len(freq) == 0:
        return None, None

    header_table = {item: [count, None] for item, count in freq.items()}
    root = FPNode('Null', 1, None)
    
    for transaction in transactions:
        transaction_items = [item for item in transaction if item in freq]
        if len(transaction_items) > 0:
            sorted_items = sorted(transaction_items, key=lambda item: header_table[item][0], reverse=True)
            update_tree(sorted_items, root, header_table, 1)
    return root, header_table

def ascend_fp_tree(node): # прохождение по дереву

    path = []
    while node.parent is not None and node.parent.item != 'Null':
        node = node.parent
        path.append(node.item)
    return path

def find_prefix_paths(base_item, header_table):
    
    conditional_patterns = {} #Словарь
    node = header_table[base_item][1]
    while node is not None:
        prefix_path = ascend_fp_tree(node)
        if len(prefix_path) > 0:
            conditional_patterns[frozenset(prefix_path)] = node.count
        node = node.node_link
    return conditional_patterns # возврат словаря, где frozenset клуч пути, а значение - счёт

def mine_fp_tree(tree, header_table, min_support, pre_fix, frequent_itemsets):
    """
    Рекурсивно извлекает частые наборы элементов из FP-дерева.
    
    tree: Текущий FP-tree (FPNode).
    header_table: Заголовочная таблица.
    min_support: Порог поддержки.
    pre_fix: Набор накопленных элементов (set).
    frequent_itemsets: Словарь для сохранения найденных наборов.
    """
    sorted_items = sorted(header_table.items(), key=lambda x: x[1][0])
    for base_item, base_info in sorted_items:
        new_freq_set = pre_fix.copy()
        new_freq_set.add(base_item)
        frequent_itemsets[frozenset(new_freq_set)] = base_info[0]
        conditional_pattern_bases = find_prefix_paths(base_item, header_table)
        conditional_transactions = []
        for path, count in conditional_pattern_bases.items():
            transaction = list(path)
            for _ in range(count):
                conditional_transactions.append(transaction)
        if len(conditional_transactions) > 0:
            conditional_tree, conditional_header = create_fp_tree(conditional_transactions, min_support)
            if conditional_header is not None:
                mine_fp_tree(conditional_tree, conditional_header, min_support, new_freq_set, frequent_itemsets)


In [ ]:
#Загрузка данных, построение FP-дерева и выбор частых наборов

df = pd.read_csv(r'C:\Users\Гребенников Матвей\Desktop\Диплом\Диплом\GeneratedLabelledFlows\TrafficLabelling\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
df.columns = df.columns.str.strip()
print (df.columns)
transactions = preprocess_data(df)

min_support = 10 #минимальный порог

# FP-tree
fp_tree, header_table = create_fp_tree(transactions, min_support)
if fp_tree is None:
    print("Нет частых элементов, удовлетворяющих минимальному порогу поддержки.")
else:
    frequent_itemsets = {}
    mine_fp_tree(fp_tree, header_table, min_support, set(), frequent_itemsets)
    
    # Вывод частых наборов, представленных как frozenset
    print("Найденные частые наборы (MDFP):")
    for itemset, support in frequent_itemsets.items():
        print(set(itemset), "поддержка:", support)



Index(['Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
  